In [5]:
import pandas as pd
pd.set_option('use_inf_as_na', True)
import numpy as np
from scipy.spatial import distance
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import rpy2
import rpy2.robjects.packages as rpackages
import rpy2.robjects as ro
pandas2ri.activate()
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.conversion import localconverter
import warnings
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)
pandas2ri.activate()
from matplotlib import pyplot
from collections import Counter
from sklearn.model_selection import StratifiedKFold, KFold
from numpy import mean
from numpy import std
import xgboost as xgb
from pandas import read_csv
from sklearn.preprocessing import RobustScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, StandardScaler
from scipy.stats import normaltest
from sklearn.model_selection import ParameterSampler
from numpy.random import randn
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from numpy import *
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats.stats import pearsonr, spearmanr
from scipy import stats
import tensorflow as tf
import multiprocessing as mp
import time
from sklearn.metrics.cluster import normalized_mutual_info_score
import os
from sklearn.svm import SVR
import collections
import matplotlib.pyplot as plt
from collections import OrderedDict
import category_encoders as ce
import itertools as it
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import re
import warnings
warnings.filterwarnings("ignore")

#specify if you wish to apply over sampling by smogn
smogn = False




def get_rare(y, method, extr_type, thresh, coef, control_pts):

    # we will be getting the relevance function on all the data not just the training data because
    # when we want to apply Lime on the 'rare' testing instances, the relevance function must map all possible demand
    # values to a certain relevance. If it happens that some demand values are present only in the testing
    # and not in the training data, we cannot detect rare values correctly. The way we compute
    # rare values depends on the relevance

    # param y: the target variable vector
    # param method: 'extremes' or 'range'. Default is 'extremes'
    # param extr_type: 'both', 'high', or 'low'
    # param thresh: threshold. Default is 0.8
    # param coef: parameter needed for method "extremes" to specify how far the wiskers extend to the most extreme data point in the boxplot. The default is 1.5.
    # param control_pts: if method == 'range', then this is the relevance matrix provided by the user. Default is None

    # return the indices of the rare values in the data

    yrel = get_relevance_2(y, df=None, target_variable=None, method=method, extr_type=extr_type, control_pts=control_pts)

    # get the the phi.control returned parameters that are used as input for computing the relevance function phi
    # (function provided by R UBL's package: https://www.rdocumentation.org/packages/UBL/versions/0.0.6/topics/phi)
    # (function provided by R UBL's package
    # https://www.rdocumentation.org/packages/UBL/versions/0.0.6/topics/phi.control)
    # we need those returned parameters for computing rare values

    print('relevance method - phi function : {}'.format(method))

    if control_pts is None:
        # without relevance matrix
        print('control.pts - phi function: {}'.format(control_pts))
        print('without relevance matrix')
        params = runit.get_relevance_params_extremes(y, rel_method=method, extr_type=extr_type, coef=coef)
    else:
        # with relevance matrix (provided by the user)
        print('control.pts - phi function: {}'.format(control_pts))
        print('with relevance matrix')
        params = runit.get_relevance_params_range(y, rel_method=method, extr_type=extr_type, coef=coef,
                                                  relevance_pts=control_pts)

    # phi params
    phi_params = params[0]
    loss_params = params[1]

    phi_params = dict(zip(phi_params.names, list(phi_params)))
    loss_params = dict(zip(loss_params.names, list(loss_params)))

    print('\nCONTROL PTS')
    print(phi_params['control.pts'])
    print("for the whole dataset")
    rare_indices = get_rare_indices(y=y, y_rel=yrel, thresh=thresh, controlpts=phi_params['control.pts'])
    # print('rare indices are: {}'.format(rare_indices))

    return rare_indices, phi_params, loss_params, yrel


def get_relevance_2(y, df, target_variable, method, extr_type, control_pts):

    # gets the relevance values of the target variable vector
    # param y: the target variable vector
    # param df: if y in None, this must be passed. It is the data frame of interest
    # param target_variable: if y is None, this must be passed. It is the name of the target variable
    # param method: 'extremes' or 'range'
    # param extr_type: 'both', 'high', or 'low'
    # param control_pts: if method == 'range', will be a relevance matrix provided by the user
    # return: the relevance values of the associated target variable

    # get the target variable vector y
    if y is None:
        if df is None or target_variable is None:
            raise ValueError('if y is None, neither df nor target_variable must be None')
        y = df[target_variable]

    # check that the passed parameters are in order
    if method != 'range' and method != 'extremes':
        raise ValueError('method must be "range" or "extremes", there is no method called "%s"' % method)
    elif method == 'range' and control_pts is None:
        raise ValueError('If method == "range", then control_pts must not be None')
    elif method == 'extremes' and extr_type not in ['high', 'low', 'both']:
        raise ValueError('extr_type must wither be "high", "low", or "both"')
    else:
        if control_pts is None:
            print('getting yrel - Control pts is {}, method is {}'.format(control_pts, method))
            y_rel = runit.get_yrel(y=np.array(y), meth=method, extr_type=extr_type)
        else:
            print('getting yrel - Control pts is not None, method is {}'.format(method))
            y_rel = runit.get_yrel(y=np.array(y), meth=method, extr_type=extr_type, control_pts=control_pts)

    return y_rel


def get_rare_indices(y, y_rel, thresh, controlpts):
    # get the indices of the rare values in the data
    # param y: the target variable vector
    # param y_rel: the target variable (y) relevance vector
    # param thresh: the threshold of interest
    # param controlpts: the phi.control (function provided by R UBL's package: https://www.rdocumentation.org/packages/UBL/versions/0.0.6/topics/phi.control)
    # returned parameters that are used as input for computing the relevance function phi (function provided by R UBL's package: https://www.rdocumentation.org/packages/UBL/versions/0.0.6/topics/phi)
    # return: the indices of the rare values in 'y'
    

    # references
    # https://github.com/paobranco/SMOGN-LIDTA17/blob/8964a2327de19f6ca9e6f7055479ca863cd6b8a0/R_Code/ExpsDIBS.R#L41

    # transform controlpts returned by R into a python list
    controlpts = list(np.array(controlpts))
    # print(controlpts)

    # boolean variable indicating whether both low and high rare exist
    both = [controlpts[i] for i in [1, 7]] == [1, 1]

    # initialize rare cases to empty list (in case there are no rare cases at all)
    rare_cases = []

    if both:
        # bothr = True
        print('\nWe have both low and high extremes')
        rare_low = [i for i, e in enumerate(y_rel) if e > thresh and y[i] < controlpts[3]]
        rare_high = [i for i, e in enumerate(y_rel) if e > thresh and y[i] > controlpts[3]]

        # merge two lists (of low rare + high rare) together
        rare_cases = rare_low + rare_high

    else:
        print('\nWe dont have both', end=' ')
        if controlpts[1] == 1:
            print('We have only low rare')
            # lowr = True
            rare_cases = [i for i, e in enumerate(y_rel) if e > thresh and y[i] < controlpts[3]]
        else:
            print('We have only high rare')
            # highr = True
            rare_cases = [i for i, e in enumerate(y_rel) if e > thresh and y[i] > controlpts[3]]

    total = len(rare_cases)

    print('Total Number of rare cases: %d out of %d' % (total, len(y)), file=open(output_path +"smogn_stats.txt", "a"))
    print('Percentage of Rare Cases: %.2f%%\n' % (total/len(y) * 100), file=open(output_path +"smogn_stats.txt", "a"))

    return rare_cases


def round_oversampled_one_hot_encoded(df):

    # round one hot encoded vectors of an oversampled dataset. We have fed the SMOGN/SMOTER/GN/RandUnder
    # a data frame having one hot encoded values (0s and 1s). However, given that we are using Euclidean/Manhattan
    # distances for oversampling, some noise is added to these making them 1.0003, 0.99, etc.
    # Having this said, this function will round these values back again so they are
    # perfect 0s or 1s. We could have used HEOM distance, but it expects "nominal" features
    # as opposed to one hot encodings.
    # param df: the over-sampled data frame
    # return: the over-sampled data frame with one hot encodings rounded

    for col in one_hot_encoded:
        df.loc[df[col] < 0.5, col] = 0
        df.loc[df[col] >=0.5, col] = 1
    return df


def count_abnormal(df):

    # Due to Oversampling, SMOGN is adding noise to the one hot encoded vectors. This function counts how many of these
    # are being done
    # param df: the oversampled data frame
    # return: statistics about the above

    count = 0
    for col in one_hot_encoded:
        for i, row in df.iterrows():
            if row[col] not in [0, 1]:
                count += 1
            else:
                continue

    print('number of noisy one hot encoded: {} out of {}'.format(count, len(df)))
    print('percentage of noisy one hot encoded: %.3f' % (count / len(df) * 100))

#calculates all error metrics needed
def calculate_errors(actual, predicted, nb_columns):
    n = len(actual)
    r2score = r2_score(actual, predicted)
    adjusted_r2 = 1 - ((1 - r2score) * (n - 1)) / (n - nb_columns - 1)
    mase = mean_absolute_error(actual, predicted)
    rms = sqrt(mean_squared_error(actual, predicted))
    mse = mean_squared_error(actual, predicted)
    re = (mse / np.mean(predicted)) * 100
    pearson, pval = stats.pearsonr(np.array(actual).ravel(), np.array(predicted).ravel())
    mae = np.mean(np.abs((actual - predicted) / actual)) * 100
    spearman_corr, _ = spearmanr(actual, predicted)
    distance_corr = distance.correlation(actual, predicted)
    mape_score = np.asarray(np.abs(( np.array(actual) - np.array(predicted)) / np.array(actual)), dtype=np.float64).mean() * 100
    return r2score, mase, rms, mse, re, pearson, pval, mae, adjusted_r2, spearman_corr, distance_corr, mape_score

#get indices of folds in Stratified KFold CV
def get_fold_indices(X,y,n_splits,rare_values):
    rare_vec = [1 if i in rare_values else 0 for i in range(len(y))]
    y = np.array(rare_vec)
    splitter = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=123)
    folds = list(splitter.split(X, y))
    return folds
    
#get indices of folds in Stratified KFold CV
def get_rare_idex(X,y,rare_values):
    rare_vec = [1 if i in rare_values else 0 for i in range(len(y))]
    y = np.array(rare_vec)
    return y
  
#get grid of all hyper-parameters
def get_param_grid(dicts):
  return [dict(zip(dicts.keys(), p)) for p in it.product(*dicts.values())]

def model_fit_predict_CV(X, y, split, parameter, model_name):

    X_train, y_train = X.iloc[split[0],:], y.iloc[split[0], :]
    X_valid, y_valid   = X.iloc[split[1],:], y.iloc[split[1], :]
    #removed param
    model = xgb.XGBRegressor(**parameter)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)

    df_test = X_valid

    # combine y_test and y_pred in 1 dataset
    df_test[y_test_name] = y_valid
    df_test[y_test_pred_name] = y_pred
    
    accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall = evaluate(df_test, actual=y_test_name, predicted=y_test_pred_name,
             thresh=0.8, rel_method='extremes', extr_type='high',
             coef=1.5, relevance_pts=None)
             
    return accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall

#make input data pipeline for tf model
def make_input_fn(X, y, n_epochs=None, shuffle=False):
    NUM_EXAMPLES = math.floor(len(y) / 2)

    def input_fn():
    
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        #print(dataset)
        if shuffle:
          dataset = dataset.shuffle(NUM_EXAMPLES)
        #For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = dataset.repeat(n_epochs)
        # In memory training doesn't use batching.
        dataset = dataset.batch(NUM_EXAMPLES,  drop_remainder=False)
        #print(dataset)
        return dataset

    return input_fn

#make test input data pipeline for tf model
def make_input_fn_test(X, n_epochs=None, shuffle=False):
    NUM_EXAMPLES = math.floor(len(X) / 2)
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices(dict(X))
        if shuffle:
            dataset = dataset.shuffle(NUM_EXAMPLES)
        # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = dataset.repeat(n_epochs)
        # In memory training doesn't use batching.
        dataset = dataset.batch(NUM_EXAMPLES,drop_remainder=True)
        #print(dataset)
        return dataset

    return input_fn



def rarify_data(df, df_train, df_test, target_variable, method, extr_type, thresh, coef, control_pts):

    # get df_train and df_test
    # param df_train: the training data frame
    # param df_test: the testing data frame
    # param target_variable: name of the target variable column
    # return: df_train and df_test with equal class distribution between classes: rare and not rare

    print("checking null values in dataset when applying rarify")
    print(df.isnull().values.any())

    # get y, reset the index to avoid falsy retrievals by index later on
    y = df[target_variable].reset_index(drop=True)
    # get the indices of the rare values in the combined data frame
    # note that the relevance returned is the relevance of the whole data frame not just the training
    rare_values, phi_params, loss_params, yrel = get_rare(y, method, extr_type,thresh, coef, control_pts)

    # dictionary mapping each value to its relevance
    demandrel = {}
    relvals = np.array(yrel)

    for i, e in enumerate(y):
        if e not in demandrel:
            rel = relvals[i]
            demandrel[e] = rel

    # now we have the indices of the rare values, get their percentage

    # percentage of rare values in the whole dataset
    prare = len(rare_values)/len(df)
    print('percentage of rare values in dataset before smogn: ' + str(prare*100) , file=open(output_path +"rare_perc_results.txt", "a"))
    # number of rare values in the whole dataset
    numrare = len(rare_values)
    print('number of rare values in dataset before smogn: {}/{}'.format(numrare, len(df)), file=open(output_path +"rare_perc_results.txt", "a"))

    # number of rare values that are be in each of the train and test
    numraretrain = int(round(prare * len(df_train)))
    numraretest = int(round(prare * len(df_test)))

    print('number of rare in train: {}/{}'.format(numraretrain, len(df_train)), file=open(output_path +"smogn_stats.txt", "a"))
    print('==> {}%%'.format((numraretrain/len(df_train))*100), file=open(output_path +"smogn_stats.txt", "a"))
    print('number of rare in test: {}/{}'.format(numraretest, len(df_test)), file=open(output_path +"smogn_stats.txt", "a"))
    print('==> {}%%'.format((numraretest / len(df_test))*100), file=open(output_path +"smogn_stats.txt", "a"))

    rare_values = sorted(rare_values)

    # rare indices partitioned for each of the train and test
    rtrain = rare_values[:numraretrain]
    rtest = rare_values[numraretrain:]

    # get the relevance of each of the  dftrain and dftest
    yreltrain = [demandrel[d] for d in df_train[target_variable]]
    yreltest = [demandrel[d] for d in df_test[target_variable]]

    if len(rtrain) != numraretrain:
        raise ValueError('Incompatibility between the number of rare values that must be included in the '
                         'training data for equal class distribution and the obtained number of rare')

    if len(rtest) != numraretest:
        raise ValueError('Incompatibility between the number of rare values that must be included in the '
                         'testing data for equal class distribution and the obtained number of rare')

    return rtrain, rtest, yreltrain, yreltest, phi_params['control.pts'], loss_params, demandrel

#required error metrics
def error_metrics(y_test, y_pred, ncols):
    r2score, mase, rms, mse, re, pearson, pval, mae, adjusted_r2, spearman_corr, distance_corr, mape = calculate_errors(y_test, y_pred, ncols)
    print("The range for the output variable is:" + str(y_test.mean()))
    print("r2score : " + str(r2score))
    print("adj r2score : " + str(adjusted_r2))
    print("mae : " + str(mase))
    print("rmse : " + str(rms))
    print("mse : " + str(mse))
    print("re : " + str(re))
    print("pearson : " + str(pearson))
    print("spearman : " + str(spearman_corr))
    print("distance : " + str(distance_corr))
    print("mape : " + str(mape))

#evaluate ub error metrics
def evaluate(df, actual, predicted, thresh, rel_method='extremes', extr_type='high', coef=1.5, relevance_pts=None):
    y = np.array(df[actual])
    phi_params, loss_params, _ = get_phi_loss_params(y, rel_method, extr_type, coef, relevance_pts)

    nb_columns = len(list(df.columns.values)) - 1

    accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall = get_stats(df[actual], df[predicted], nb_columns, thresh, phi_params, loss_params)
    return accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall


def get_phi_loss_params(y, rel_method, extr_type='high', coef=1.5, relevance_pts=None):

    # get the parameters of the relevance function
    # param df: dataframe being used
    # param target_variable: name of the target variable
    # param rel_method: either 'extremes' or 'range'
    # param extr_type: either 'high', 'low', or 'both' (defualt)
    # param coef: default: 1.5
    # param relevance_pts: the relevance matrix in case rel_method = 'range'
    # return: phi parameters and loss parameters


    if relevance_pts is None:
        print('Will not use relevance matrix')
        params = runit.get_relevance_params_extremes(y, rel_method=rel_method, extr_type=extr_type, coef=coef)
    else:
        print('Using supplied relevance matrix')
        params = runit.get_relevance_params_range(y, rel_method=rel_method, extr_type=extr_type, coef=coef,
                                                  relevance_pts=relevance_pts)

    # phi params and loss params
    phi_params = params[0]
    loss_params = params[1]
    relevance_values = params[2]

    phi_params = dict(zip(phi_params.names, list(phi_params)))
    loss_params = dict(zip(loss_params.names, list(loss_params)))

    return phi_params, loss_params, relevance_values


def get_stats(y_test, y_pred, nb_columns, thr_rel, phi_params, loss_params):

    # Function to compute regression error metrics between actual and predicted values +
    # correlation between both using different methods: Pearson, Spearman, and Distance
    # param y_test: the actual values. Example df['actual'] (the string inside is the name
    # of the actual column. Example: df['LE (mm)'], df['demand'], etc.)
    # param y_pred: the predicted vlaues. Example df['predicted']
    # param nb_columns: number of columns <<discarding the target variable column>>
    # return: R2, Adj-R2, RMSE, MSE, MAE, MAPE

    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    if not isinstance(y_test, list):
        y_test = list(y_test)
    if not isinstance(y_pred, list):
        y_pred = list(y_pred)

    n = len(y_test)

    r2_Score = r2_score(y_test, y_pred)  # r-squared
    adjusted_r2 = 1 - ((1 - r2_Score) * (n - 1)) / (n - nb_columns - 1)  # adjusted r-squared
    rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))  # RMSE
    mse_score = mean_squared_error(y_test, y_pred)  # MSE
    mae_score = mean_absolute_error(y_test, y_pred)  # MAE
    #print(np.asarray(np.abs(( np.array(y_test) - np.array(y_pred)) / np.array(y_test)), dtype=np.float64))
    mape_score = np.asarray(np.abs(( np.array(y_test) - np.array(y_pred)) / np.array(y_test)), dtype=np.float64).mean() * 100  # MAPE
    accuracy = 100 - mape_score
    aic = len(y_test) * np.log(mse_score)
    bic = len(y_test) * np.log(mse_score)
    nmi = normalized_mutual_info_score(y_test, y_pred)

    trues = np.array(y_test)
    preds = np.array(y_pred)

    method = phi_params['method']
    npts = phi_params['npts']
    controlpts = phi_params['control.pts']
    ymin = loss_params['ymin']
    ymax = loss_params['ymax']
    tloss = loss_params['tloss']
    epsilon = loss_params['epsilon']

    rmetrics = runit.eval_stats(trues, preds, thr_rel, method, npts, controlpts, ymin, ymax, tloss, epsilon)

    # create a dictionary of the r metrics extracted above
    rmetrics_dict = dict(zip(rmetrics.names, list(rmetrics)))

    if isinstance(y_pred[0], np.ndarray):
        y_pred_new = [x[0] for x in y_pred]
        y_pred = y_pred_new
    
    pearson_corr, _ = pearsonr(y_test, y_pred)
    spearman_corr, _ = spearmanr(y_test, y_pred)
    distance_corr = distance.correlation(y_test, y_pred)

    print('\nUtility Based Metrics')
    print('F1: %.5f' % rmetrics_dict['ubaF1'][0])
    print('F2: %.5f' % rmetrics_dict['ubaF2'][0])
    print('F05: %.5f' % rmetrics_dict['ubaF05'][0])
    print('precision: %.5f' % rmetrics_dict['ubaprec'][0])
    print('recall: %.5f' % rmetrics_dict['ubarec'][0])

    print('\nRegression Error Metrics')
    print('R2: %.5f' % r2_Score)
    print('Adj-R2: %.5f' % adjusted_r2)
    print('RMSE: %.5f' % rmse_score)
    print('MSE: %.5f' % mse_score)
    print('MAE: %.5f' % mae_score)
    print('MAPE: %.5f' % mape_score)
    print('Accuracy: %.5f' % accuracy)
    print('aic: %.5f' % aic)
    print('bic: %.5f' % bic)
    print('nmi: %.5f' % nmi)

    print('\nCorrelations')
    print('Pearson: %.5f' % pearson_corr)
    print('Spearman: %.5f' % spearman_corr)
    print('Distance: %.5f' % distance_corr)
    
    print('TO EXCEL')
    print(np.average(y_test))
    print(rmetrics_dict['ubaF1'][0])
    print(rmetrics_dict['ubaF2'][0])
    print(rmetrics_dict['ubaF05'][0])
    print(rmetrics_dict['ubaprec'][0])
    print(rmetrics_dict['ubarec'][0])
    print(r2_Score)
    print(adjusted_r2)
    print(rmse_score)
    print(mse_score)
    print(mae_score)
    print(mape_score)
    print(accuracy)
    print(aic)
    print(bic)
    print(nmi)

    print(pearson_corr)
    print(spearman_corr)
    print(distance_corr)
    return accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,rmetrics_dict['ubaF1'][0],rmetrics_dict['ubaF2'][0],rmetrics_dict['ubaF05'][0],rmetrics_dict['ubaprec'][0],rmetrics_dict['ubarec'][0]
    
    
def calculate_avg_error_metrics(mape_folds,  acc_folds, aic_folds, bic_folds, nmi_folds, d_f,sp_f, p_f, mae_f,mse_f, rmse_f, ar2_f, r2_f, f1_f, f2_f, f5_f,prec_f,recall_f,folds):
  print('\nUtility Based Metrics Across All', file=open(output_path +"output_CV_results.txt", "a"))
  avg_f1 = f1_f / folds
  print('F1: ' , avg_f1, file=open(output_path +"output_CV_results.txt", "a"))
  avg_f2 =  f2_f / folds
  print('F2: ' , avg_f2, file=open(output_path +"output_CV_results.txt", "a") )
  avg_f5 = f5_f / folds
  print('F05:' ,  avg_f5, file=open(output_path +"output_CV_results.txt", "a"))
  avg_prec = prec_f / folds
  print('precision: ', avg_prec  , file=open(output_path +"output_CV_results.txt", "a"))
  avg_recall = recall_f / folds
  print('recall:' , avg_recall , file=open(output_path +"output_CV_results.txt", "a"))
  
  print('\nRegression Error Metrics Across All', file=open(output_path +"output_CV_results.txt", "a"))
  avg_r2 = r2_f/folds
  print('R2:' , avg_r2, file=open(output_path +"output_CV_results.txt", "a"))
  avg_ar2 =  ar2_f/folds
  print('Adj-R2:' , avg_ar2, file=open(output_path +"output_CV_results.txt", "a"))
  avg_rmse = rmse_f/folds
  print('RMSE:' , avg_rmse , file=open(output_path +"output_CV_results.txt", "a"))
  avg_mse = mse_f/folds
  print('MSE:' , avg_mse , file=open(output_path +"output_CV_results.txt", "a"))
  avg_mae =  mae_f/folds
  print('MAE:' , avg_mae, file=open(output_path +"output_CV_results.txt", "a"))
  avg_mape = mape_folds/folds
  print('MAPE:' , avg_mape , file=open(output_path +"output_CV_results.txt", "a"))
  avg_acc = acc_folds/folds
  print('Accuracy:' , avg_acc , file=open(output_path +"output_CV_results.txt", "a"))
  avg_aic = aic_folds/folds
  print('AIC:' , avg_aic , file=open(output_path +"output_CV_results.txt", "a"))
  avg_bic = bic_folds/folds
  print('BIC:' , avg_bic , file=open(output_path +"output_CV_results.txt", "a"))
  avg_nmi = nmi_folds/folds
  print('NMI:' , avg_nmi , file=open(output_path +"output_CV_results.txt", "a"))


  print('\nCorrelations Across All', file=open(output_path +"output_CV_results.txt", "a"))
  avg_pearson =  p_f/folds
  print('Pearson:' , avg_pearson, file=open(output_path +"output_CV_results.txt", "a"))
  avg_spearman = sp_f/folds
  print('Spearman:' , avg_spearman , file=open(output_path +"output_CV_results.txt", "a"))
  avg_dist =  d_f/folds
  print('Distance:' , avg_dist, file=open(output_path +"output_CV_results.txt", "a"))
  return avg_f1, avg_f2, avg_f5, avg_prec, avg_recall, avg_r2, avg_ar2, avg_rmse, avg_mse, avg_mae, avg_mape, avg_acc, avg_aic, avg_bic, avg_nmi, avg_pearson, avg_spearman, avg_dist
  
def get_relevance_oversampling(smogned, target_variable, targetrel):

    # gets the relevance values of an oversampled data frame
    # param smogned: the oversampled data frame
    # param target_variable: name of the target variable column
    # param targetrel: dictionary mapping each target variable value to a relevance value
    # return: the relevance of the oversampled data frame

    yrelafter = []
    distances = []
    for val in smogned[target_variable]:
        if val in targetrel:
            yrelafter.append(targetrel[val])
        else:
            nearest = min(sorted(list(targetrel.keys())), key=lambda x: abs(x - val))
            distances.append(abs(nearest - val))
            yrelafter.append(targetrel[nearest])

    return yrelafter, distances
  
def get_formula(target_variable):

    # gets the formula for passing it to R functions. Example: target_variable ~ col1 + col2 ...
    # param target_variable: the name of the target variable
    # return: R's formula as follows: target_variable ~ other[0] + other[1] + other[2] + other[3] + ...

    formula = runit.create_formula(target_variable)
    return formula
    
def apply_smogn(df_train, smogn, target_variable, phi_params, thr_rel, Cperc, k, repl, dist, p, pert, plotdensity=False ):

    #method that applies SMOGN Algorithm to the current data frame
    # print('getting back values from oversampled R data frame')
    # print('before smogn')
    #print(pandas2ri.py2ri(df_train).head(), "this is py2ri")
    if smogn:
        smogned = runit.WFDIBS(
            fmla=get_formula(target_variable),
            dat= pandas2ri.py2ri(df_train),
            #dat=df_train,
            method=phi_params['method'][0],
            npts=phi_params['npts'][0],
            controlpts=phi_params['control.pts'],
            thrrel=thr_rel,
            Cperc=Cperc,
            k=k,
            repl=repl,
            dist=dist,
            p=p, 
            pert=pert)

        # print('after smogn')
        # print('before pandas2ri')
         #convert the oversampled R Data.Frame back to a pandas data frame
        smogned = pandas2ri.ri2py_dataframe(smogned)
        # print('after pandas2ri')

        if plotdensity:
            # density plot after smooting
            plot_density(smogned,target_variable,output_folder + 'plots/', 'density_after_smogn', 'Density Plot')

        X_train = np.array(smogned.loc[:, smogned.columns != target_variable])
        y_train = np.array(smogned.loc[:, target_variable])

        return X_train, y_train
  
def write_to_txt(filename, content):
  text_file = open(output_path + filename, "w")
  text_file.write(content)
  text_file.close()
  
def plot_actual_vs_predicted(df, predicted_variable):
  plt.plot(list(range(1, len(df) + 1)), df[y_test_name], color='b', label='actual')
  plt.plot(list(range(1, len(df) + 1)), df[predicted_variable], color='r', label='predicted')
  plt.legend(loc='best')
  plt.suptitle('actual vs. predicted')
  plt.savefig(output_path + 'actual_vs_predicted')
  plt.close()
    
def plot_actual_vs_predicted_scatter_bisector(df, predicted_variable):
  fig, ax = plt.subplots()
  ax.scatter(df[y_test_name], df[predicted_variable], c='black')
  lims = [
      np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
      np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
  ]
  ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
  ax.set_aspect('equal')
  ax.set_xlim(lims)
  ax.set_ylim(lims)
  plt.suptitle('actual vs. predicted forecasts')
  plt.savefig(output_path + 'actual_vs_predicted_scatter_plot')
  plt.close()


def plot_relevance(y, yrel, target_variable, output_folder, fig_name):
    reldict = {}
    y = y[target_variable]
    for i, e in enumerate(y):
        if e not in reldict:
            reldict[e] = yrel[i]

    reldict = dict(collections.OrderedDict(sorted(reldict.items())))
    plt.plot(list(reldict.keys()), list(reldict.values()))
    plt.xlabel(target_variable)
    plt.ylabel('relevance')

    plt.savefig(output_folder + fig_name)
    plt.close()

def plot_target_variable(df, df_resampled, output_column, output_folder, fig_name):
    y = df[output_column]
    y_resamp = df_resampled[output_column]
    plt.plot(list(range(len(y))), sorted(y), label = "original")
    plt.plot(list(range(len(y_resamp))), sorted(y_resamp), label = "resampled")
    plt.xlabel('Index')
    plt.ylabel(target_variable)
    plt.legend()
    plt.savefig(output_folder + fig_name)
    plt.close()
  
def get_relevance():
    ctrl = phi_params['control.pts']
    if rel_method[0] == 'extremes' and relevance_pts[0] is None:
        rell = np.array([
            [ctrl[0], ctrl[1], ctrl[2]],
            [ctrl[3], ctrl[4], ctrl[5]],
            [ctrl[6], ctrl[7], ctrl[8]]
        ])
    else:
        rell = relevance_pts[0]

    return rell

## Generate lags for all input features, re-generate even if some exist so that order will not be shuffled after nan dropping
def generate_lags_for(df, column, lags_count):
        for i in range(lags_count):
            lag_name = column + "-" + str(i + 1)
            df[lag_name] = df[column].shift(i + 1)
        return df

def generate_lags(df, lagsForColumns):
    '''This function generates the lags for the list of columns'''
    for k in range(len(lagsForColumns)):
        col = lagsForColumns[k]
        if col in df.columns:
            df = generate_lags_for(df, col, 5)
    return df


def split_train_test_valid(df, TRAIN_RATIO, TEST_RATIO):
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()
    Y_train = pd.DataFrame()
    Y_test = pd.DataFrame()
    unique_sites = df["Site"].unique()
    print("Number of sites:", len(unique_sites))

    for site in unique_sites:
        df_site = df[df["Site"] == site]
        X = df_site
        train_index = int(X.shape[0] * TRAIN_RATIO)
        test_index = int(X.shape[0] * (TRAIN_RATIO + TEST_RATIO))

        X_train = X_train.append(X[:train_index], ignore_index = True)
        X_test = X_test.append(X[train_index:], ignore_index = True)
        Y_train = Y_train.append(X[:train_index], ignore_index = True)
        Y_test = Y_test.append(X[train_index:], ignore_index = True)

    Y_train = Y_train[[output_column]]
    Y_test = Y_test[[output_column]]
   
    X_train = X_train.drop([output_column], axis = 1)
    X_test = X_test.drop([output_column], axis = 1)
   
    return X_train, X_test, Y_train, Y_test

def binary_encode_column(df, columnToEncode):
    encoder = ce.BinaryEncoder(cols=[columnToEncode])
    df_encoder = encoder.fit_transform(df[columnToEncode])
    df = pd.concat([df, df_encoder], axis=1)
    return df

def export_scores(scores, columnName):
    file_name = output_path + "score_sheet_final.csv"
    if not os.path.exists(file_name):
        print("path does not exist")
        df = pd.DataFrame(list())
        df.to_csv(file_name, index=False)
    else:
        print("path exists")
        df = pd.read_csv(file_name, delimiter=',')

    df["Error Metrics"] = scores.keys()
    df[columnName] = scores.values()
    df.to_csv(file_name, index=False)
    return df


########################################################################################################################
                                            #Establish a connection to R library
########################################################################################################################
rpy2.robjects.numpy2ri.activate()
runit = robjects.r
runit['source']('smogn.R')


########################################################################################################################
                                            #Read and Preprocess Dataset
########################################################################################################################

#smogn relate hyper-params
target_variable = "LE_bowen_corr_mm"
rel_method='range'
extr_type='high'
coef=1.5

rell = np.array([
    [1, 0 , 0],
    [4, 0 , 0],
    [15, 1 , 0],
])
#rell = None
relevance_pts=rell
rel="auto"
thr_rel=0.1
Cperc=np.array([1,1.2])
k=5
repl=False
dist="Manhattan"
p=2
pert=0.1
input_path = "/apps/data/output_eeflux.csv"

#read csv
df_test = pd.read_csv(input_path, delimiter=',')

df_test.loc[(df_test['Eeflux_ET'] < 0.3 ), 'Eeflux_ET'] = 0.3
df_test.loc[(df_test['Eeflux_ET'] > 15), 'Eeflux_ET'] = 15

df_test['resid1'] = df_test['LE_bowen_corr_mm']
df_test['resid2'] =  df_test['Eeflux_ET']

df_test['resid3'] = df_test['Eeflux_ET'] / df_test['LE_bowen_corr_mm']
df_test['resid4'] =  df_test['Eeflux_ET'] / df_test['LE_bowen_corr_pred']

df_test['resid5'] = df_test['Eeflux_ET'] - df_test['LE_bowen_corr_mm']
df_test['resid6'] = df_test['Eeflux_ET'] - df_test['LE_bowen_corr_pred']

accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall = evaluate(df_test, actual='resid1', predicted='resid2',
        thresh=0.1, rel_method=rel_method, extr_type='high',
        coef=1.5, relevance_pts=rell)


accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall = evaluate(df_test, actual='resid3', predicted='resid4',
        thresh=0.1, rel_method=rel_method, extr_type='high',
        coef=1.5, relevance_pts=rell)

accuracy, aic, bic, nmi, mape_score,distance_corr,spearman_corr,pearson_corr,mae_score,mse_score,rmse_score,adjusted_r2,r2_Score,f1,f2,f5,prec,recall = evaluate(df_test, actual='resid5', predicted='resid6',
        thresh=0.1, rel_method=rel_method, extr_type='high',
        coef=1.5, relevance_pts=rell)

Using supplied relevance matrix

Utility Based Metrics
F1: 0.79518
F2: 0.80594
F05: 0.78471
precision: 0.77788
recall: 0.81328

Regression Error Metrics
R2: -1.15716
Adj-R2: -4.63258
RMSE: 3.02911
MSE: 9.17552
MAE: 2.11303
MAPE: 56.70533
Accuracy: 43.29467
aic: 210.57120
bic: 210.57120
nmi: 0.89147

Correlations
Pearson: 0.26641
Spearman: 0.27604
Distance: 0.73359
TO EXCEL
3.806324413789474
0.7951821078007248
0.8059403451553143
0.7847073034615862
0.7778760802231185
0.8132756988367532
-1.157159975250167
-4.63258437981988
3.0291119774417083
9.175519371880815
2.1130309381684205
56.705329907124636
43.294670092875364
210.57120496465657
210.57120496465657
0.8914699840889503
0.2664075411028962
0.2760414205202841
0.7335924588971039
Using supplied relevance matrix

Utility Based Metrics
F1: 0.00001
F2: 0.00001
F05: 0.00001
precision: 0.00001
recall: 0.00001

Regression Error Metrics
R2: 0.66021
Adj-R2: 0.11277
RMSE: 0.45671
MSE: 0.20858
MAE: 0.25173
MAPE: 29.01079
Accuracy: 70.98921
aic: -148.9